# Indicators descriptives statistics

In [1]:
import time
import os 
os.chdir('../../')

import tqdm
import yaml 
import pandas as pd

from package.indicators.utils import * 
from package import Dataset

with open("mongo_config.yaml", "r") as infile:
    pars = yaml.safe_load(infile)['PC_BETA']
    
client = pymongo.MongoClient(pars['client_name'])
db = client[pars['db_name']]
collection = db[pars['pkg']['collection_name']]

In [2]:
for year in range(2000,2006):
    docs = collection.find({pars['pkg']['year_var']:year})
    docs_list = [doc for doc in tqdm.tqdm(docs) if 'journal_commonness_score' in doc.keys()]
    df = pd.DataFrame(docs_list)
    df.to_csv('Data/test_indicator_df_'+str(year)+'.csv')

488363it [05:33, 1462.66it/s]
498691it [21:38, 384.20it/s] 
513695it [20:37, 415.07it/s]
536511it [31:10, 286.86it/s]
571260it [29:04, 327.46it/s] 
603464it [22:53, 439.50it/s]
